In [ ]:
import os
import re
import json
from collections import Counter

# Build a function-domain mapping dictionary
def build_function_domain_map(root_folder):
    function_domain_map = {}
    
    for domain_folder in os.listdir(root_folder):
        domain_path = os.path.join(root_folder, domain_folder)
        if os.path.isdir(domain_path):
            for subdir, _, files in os.walk(domain_path):
                for file in files:
                    if file == "api.py":
                        api_file_path = os.path.join(subdir, file)
                        with open(api_file_path, "r") as f:
                            content = f.read()
                            # Find all functions defined in api.py and map them to the domain folder
                            matches = re.findall(r"def\s+(\w+)\s*\(", content)
                            for function_name in matches:
                                function_domain_map[function_name] = domain_folder
    return function_domain_map

# Add domain field to tools using the function-domain map
def add_domain_to_tools(data, function_domain_map):
    for entry in data:
        tools = json.loads(entry.get("tools", "[]"))  # Parse the tools JSON
        for tool in tools:
            function_name = tool["name"]
            domain = function_domain_map.get(function_name, "Other")
            tool["domain"] = domain  # Add the domain key-value pair
        entry["tools"] = json.dumps(tools)  # Convert tools back to a JSON string
    return data

# Load dataset and update with domains
def process_dataset(file_path, root_folder, output_path):
    # Build the function-domain map once
    function_domain_map = build_function_domain_map(root_folder)
    
    with open(file_path, "r") as f:
        data = json.load(f)
    
    # Add domain information to each tool in each entry
    updated_data = add_domain_to_tools(data, function_domain_map)
    
    # Write updated dataset to a new JSON file
    with open(output_path, "w") as f:
        json.dump(updated_data, f, indent=4)
    
    print(f"Updated dataset with domains saved to {output_path}")

root_folder = "/Users/cho-seonggeun/Downloads/data/toolenv/tools"  
input_file_path = "data/xlam_fc.json" 
output_file_path = "data/xlam_fc_w_domain.json" 

process_dataset(input_file_path, root_folder, output_file_path)

Updated dataset with domains saved to data/xlam_fc_w_domain.json


In [107]:
with open(output_file_path, "r") as f:
    data = json.load(f)

domain_counter = {}
tool_list = []
for entry in data:
    for tool in json.loads(entry["tools"]):
        tool_name, domain = tool["name"], tool["domain"]
        if domain not in domain_counter:
            domain_counter[domain] = [tool_name]
        else:
            if tool_name not in domain_counter[domain]:
                domain_counter[domain].append(tool_name)

        if tool_name not in tool_list:
            tool_list.append(tool_name)

for domain in domain_counter:
    print(domain, ":", len(domain_counter[domain]))

Gaming : 99
Financial : 53
Finance : 398
Entertainment : 146
Sports : 358
Media : 97
Commerce : 17
Movies : 33
Weather : 78
Other : 197
Data : 637
eCommerce : 63
Travel : 108
Location : 106
Social : 212
Science : 32
Advertising : 6
Business_Software : 25
News_Media : 67
Events : 22
Business : 67
Video_Images : 42
Food : 61
Search : 36
Medical : 11
Music : 79
Education : 54
Text_Analysis : 28
Database : 76
Tools : 101
Devices : 14
Logistics : 14
Health_and_Fitness : 36
Email : 36
Communication : 29
Monitoring : 15
Artificial_Intelligence_Machine_Learning : 22
Mapping : 43
SMS : 12
Transportation : 44
Visual_Recognition : 7
Translation : 7
Energy : 5
Jobs : 5
Payments : 2
Cybersecurity : 3
Storage : 2
